## Set up network and install software

In [ ]:
from state import *
from IPython.core.getipython import get_ipython

# Import libraries
%run ../setup/include/include_libraries.py
# Import selected slice name
%run ../slice_info/selected_slice.py


# Import topology_variables

# Use the button in the GUI automatically does it for you
if os.getenv('SELECTED_SLICE') is not None:
    SELECTED_SLICE = os.getenv('SELECTED_SLICE')

# If you manually run the notebook, please specify the slice name in the line below
# SELECTED_SLICE = 'slice_name'

path = f'../slice_info/{SELECTED_SLICE}/topology_variables.ipynb'
get_ipython().run_line_magic('run', path)

import write_value


## Configure nodes

In [ ]:
success = True
try:
    slice = fablib.get_slice(name=selected_slice)
    config_threads = {}
    host_config_script = "sudo apt-get update -qq && sudo apt-get install -qq -y python3-scapy && git clone https://github.com/p4lang/tutorials.git/" 
    h1 = slice.get_node(name=h1_name)        
    if type(ip_address(h1.get_management_ip())) is IPv6Address:
        h1.execute("sudo sed -i '/nameserver/d' /etc/resolv.conf && sudo sh -c 'echo nameserver 2a00:1098:2c::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a01:4f8:c2c:123f::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a00:1098:2b::1 >> /etc/resolv.conf'")
       
    h1_os_iface = h1.get_interface(network_name=net_h1_name)
    h1_os_iface.ip_addr_add(addr=h1_addr, subnet=h1_subnet)
    h1_config_thread = h1.execute_thread(host_config_script)
    config_threads[h1] = h1_config_thread

    
    h2 = slice.get_node(name=h2_name)
    if type(ip_address(h2.get_management_ip())) is IPv6Address:
        h2.execute("sudo sed -i '/nameserver/d' /etc/resolv.conf && sudo sh -c 'echo nameserver 2a00:1098:2c::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a01:4f8:c2c:123f::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a00:1098:2b::1 >> /etc/resolv.conf'")
 
    h2_os_iface = h2.get_interface(network_name=net_h2_name)
    h2_os_iface.ip_addr_add(addr=h2_addr, subnet=h2_subnet)
    h2_config_thread = h2.execute_thread(host_config_script)
    config_threads[h2] = h2_config_thread

    h3 = slice.get_node(name=h3_name)
    if type(ip_address(h3.get_management_ip())) is IPv6Address:
        h3.execute("sudo sed -i '/nameserver/d' /etc/resolv.conf && sudo sh -c 'echo nameserver 2a00:1098:2c::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a01:4f8:c2c:123f::1 >> /etc/resolv.conf' && sudo sh -c 'echo nameserver 2a00:1098:2b::1 >> /etc/resolv.conf'")
 
    h3_os_iface = h3.get_interface(network_name=net_h3_name)
    h3_os_iface.ip_addr_add(addr=h3_addr, subnet=h3_subnet)
    h3_config_thread = h3.execute_thread(host_config_script)
    config_threads[h3] = h3_config_thread


    
except Exception as e:
    print(f"Exception: {e}")
    success = False
    raise SystemExit("Stopping notebook execution due to exception")

## Configure switches

In [ ]:
try:
    s1 = slice.get_node(name=s1_name)
    s1_h1_os_iface = s1.get_interface(network_name=net_h1_name)
    s1_s2_os_iface = s1.get_interface(network_name=net_s1_s2_name)
    s1_s3_os_iface = s1.get_interface(network_name=net_s1_s3_name)
        
    if type(ip_address(s1.get_management_ip())) is IPv4Address:
        management_ip_type = "IPv4"
    else:
        management_ip_type = "IPv6"
    
    file_attributes = s1.upload_file('./scripts/router_setup_p4_bmv2_container.sh','router_setup_p4_bmv2_container.sh')
    command=f"chmod +x router_setup_p4_bmv2_container.sh && sudo sh -c './router_setup_p4_bmv2_container.sh  {management_ip_type} {s1_h1_os_iface.get_os_interface()} {s1_s2_os_iface.get_os_interface()} {s1_s3_os_iface.get_os_interface()}  > /tmp/script.log 2>&1'"
    s1_config_thread = s1.execute_thread(command)
    config_threads[s1] = s1_config_thread
    
    
    
    s2 = slice.get_node(name=s2_name)
    s2_h2_os_iface = s2.get_interface(network_name=net_h2_name)
    s2_s1_os_iface = s2.get_interface(network_name=net_s1_s2_name)
    s2_s3_os_iface = s2.get_interface(network_name=net_s2_s3_name)

    if type(ip_address(s2.get_management_ip())) is IPv4Address:
        management_ip_type = "IPv4"
    else:
        management_ip_type = "IPv6"
    
    file_attributes = s2.upload_file('scripts/router_setup_p4_bmv2_container.sh','router_setup_p4_bmv2_container.sh')
    command= f"chmod +x router_setup_p4_bmv2_container.sh && sudo sh -c './router_setup_p4_bmv2_container.sh  {management_ip_type} {s2_h2_os_iface.get_os_interface()} {s2_s1_os_iface.get_os_interface()} {s2_s3_os_iface.get_os_interface()}  > /tmp/script.log 2>&1'"
    s2_config_thread = s2.execute_thread(command)
    config_threads[s2] = s2_config_thread
    
    s3 = slice.get_node(name=s3_name)
    s3_h3_os_iface = s3.get_interface(network_name=net_h3_name)
    s3_s1_os_iface = s3.get_interface(network_name=net_s1_s3_name)
    s3_s2_os_iface = s3.get_interface(network_name=net_s2_s3_name)

    if type(ip_address(s3.get_management_ip())) is IPv4Address:
        management_ip_type = "IPv4"
    else:
        management_ip_type = "IPv6"
     
    file_attributes = s3.upload_file('scripts/router_setup_p4_bmv2_container.sh','router_setup_p4_bmv2_container.sh')
    command = f"chmod +x router_setup_p4_bmv2_container.sh && sudo sh -c './router_setup_p4_bmv2_container.sh  {management_ip_type} {s3_h3_os_iface.get_os_interface()} {s3_s1_os_iface.get_os_interface()} {s3_s2_os_iface.get_os_interface()} > /tmp/script.log 2>&1'"
    s3_config_thread = s3.execute_thread(command)
    config_threads[s3] = s3_config_thread
    
    
    for node, thread in config_threads.items():
        stdout, stderr = thread.result()
        print(f"Config thread node {node.get_name()} complete")
        print(f"stdout: {stdout}")
        print(f"stderr: {stderr}")
    


except Exception as e:
    success = False
    print(f"Error: {e}")

## Configure p4 switch tables

In [ ]:
try:
    for switch_name in [s1_name, s2_name, s3_name]:
        switch_node = slice.get_node(name=switch_name)
        management_ip_switch = str(switch_node.get_management_ip())
        print("Swtitch Name        : {}".format(switch_node.get_name()))
        print("Management IP    : {}".format(management_ip_switch))
    
        #Configure P4 Tables
        cmd_file=f'{switch_name}_commands.txt'
        print(cmd_file)
        file_attributes = switch_node.upload_file(f'scripts/{cmd_file}',cmd_file)
        print("file_attributes: {}".format(file_attributes))

        stdout = switch_node.execute(f"sudo sh -c 'cat {cmd_file} | docker exec -i fabric_p4 simple_switch_CLI'")
        print("stdout: {}".format(stdout))
except Exception as e:
    success = False
    print(f"Error: {e}")

## Update state

In [ ]:
if success:
    update_state_local(selected_slice, 'SLICE_CONFIGURED')
    update_state_in_fim(selected_slice, read_state_from_local(selected_slice))
else:
    print ('State is not updated due to error.')